In [ ]:
%config Completer.use_jedi = False # To make auto-complete faster

#Reloads imported files automatically
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('../')

In [ ]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:87% !important; }</style>"))

In [ ]:
import pandas as pd
import numpy as np
import os

from shapely.geometry import MultiPoint, Point, Polygon

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import matplotlib.ticker as ticker
import matplotlib.cm as cm
from matplotlib import colormaps as mplcmaps

from plotting.matplotlib_param_funcs import set_matplotlib_params,reset_rcParams
set_matplotlib_params()

In [ ]:
import src.compute_variables as CV
import src.bootstrap_errors as bootstrap
import src.montecarlo_errors as monte_carlo
from src.errorconfig import MonteCarloConfig,BootstrapConfig

import utils.miscellaneous_functions as MF
import utils.load_data as load_data

import plotting.mixed_plots as MP
import plotting.plotting_helpers as PH
import plotting.map_functions as mapf

import plotting.dynamical_binning as dyn

In [ ]:
coolwarm = mplcmaps["coolwarm"]

In [ ]:
full_map_string_list,divergent_map_list = mapf.get_map_string_lists()

# Load

In [ ]:
general_path = "/Users/Luismi/Desktop/MRes_UCLan/"

In [ ]:
# obs_errors = True
obs_errors = False

GSR=True
# GSR=False

zabs = True
# zabs = False

R0 = 8.1

In [ ]:
data_path = general_path+"data/Observational_data/"

data = load_data.load_and_process_data(data_path=data_path, error_bool=obs_errors, zabs=zabs, R0=R0, GSR=GSR)

In [ ]:
dummy_df = pd.DataFrame([[1,2,3],[2,3,1],[6,3,4]], columns=['a','b','c'])

# Spatial cuts

In [ ]:
x_variable = "l"
y_variable = "b"

# extra_variable = "d"
extra_variable = "R"

In [ ]:
# original limits
original_xmin,original_xmax = data[x_variable].min(),data[x_variable].max()
original_ymin,original_ymax = data[y_variable].min(),data[y_variable].max()
original_extramin,original_extramax = data[extra_variable].min(),data[extra_variable].max()

print("Original limits:")
print(x_variable,(original_xmin,original_xmax)); print(y_variable,(original_ymin,original_ymax)); print(extra_variable,(original_extramin,original_extramax))

In [ ]:
xmin = -11
xmax = 11

ymin = 0 if zabs else -13
ymax = 13

extra_min = 0
extra_max = 3.5
# extra_max = original_extramax

print(x_variable,(xmin,xmax)); print(y_variable,(ymin,ymax)); print(extra_variable,(extra_min,extra_max))

extra_var_string = ""
if extra_min != original_extramin or extra_max != original_extramax:
    extra_var_string = f"_{extra_min if extra_min != original_extramin else ''}{extra_variable}{extra_max if extra_max != original_extramax else ''}"

In [ ]:
R_shift = 0.1
# R_shift = None

xymax = 3.5
zmax = 3

# save_bool = True
save_bool = False

if True:
    
    aspect_ratio = 1 + zmax/(2*xymax)    
    fsize = 8
    
    fig,axs=plt.subplots(figsize=(fsize,aspect_ratio*fsize),nrows=2,sharex=True,gridspec_kw={"hspace":0})#,"height_ratios":{1,zmax/xymax}})

    dmax = extra_max if extra_variable=="d" else R0+3.5
    R_max = extra_max if extra_variable=="R" else 3.5
    plot_cuts = {"d":[dmax],x_variable:[xmin,xmax],y_variable:[0,ymax],"R":[0,R_max]}
    filename, c = MP.visualise_bulge_selection(cuts_dict=plot_cuts,given_axs=axs)
    
    axs[0].scatter(data["x"],data["y"],s=0.3,color="grey")
    axs[1].scatter(data["x"],data["z"],s=0.3,color="grey")
    
    if R_shift is not None:
        MP.plot_circle(axs[0],radius=3.5,centre=[R_shift,0],color="r",label=r"$(R_\mathrm{GC}-%s)\leq %s$ kpc"%(R_shift,3.5),lw=1)
        axs[1].axvline(x=-3.5+R_shift,color="red")
        axs[1].axvline(x=3.5+R_shift,color="red")
        
    for ax in axs:
        ax.legend()
        ax.set_aspect("equal")
        
    axs[0].set(xlim=(-xymax,data["x"].max()), ylim = (-xymax,xymax))
    axs[1].set(xlim=(-xymax,data["x"].max()), ylim=(0,zmax))
    
    if True: # filename and saving
        if R_shift is not None:
            filename += f"_Rshift{R_shift}"

        if save_bool:
            if os.path.isdir("graphs/other_plots/visualise_bulge_cuts/"):
                save_path = "graphs/other_plots/visualise_bulge_cuts/"
            else:
                raise ValueError("Save path not specified")

            for fileformat in [".png"]:
                plt.savefig(save_path+filename+fileformat, dpi=200,bbox_inches="tight")
                print("Saved",save_path+filename+fileformat)
        else:
            print(filename)
    
    plt.show()

In [ ]:
data_spatial = data[(data[x_variable]>=xmin)&(data[x_variable]<=xmax)&(data[y_variable]>=ymin)&(data[y_variable]<=ymax)&(data[extra_variable]>=extra_min)&(data[extra_variable]<extra_max)]

if len(data_spatial) != len(data):
    print("Excluded",len(data)-len(data_spatial),"stars")
    print(len(data_spatial),"stars left out of the initial",len(data))

# Metallicity cuts

In [ ]:
# metal_trim = float("-inf")
metal_trim = -1

data_trim = data_spatial[data_spatial['FeH']>=metal_trim]

print(f"Chose minimum metallicity of {metal_trim}" if metal_trim != float("-inf") else "No minimum metallicity")
print(len(data_trim),"stars left")

In [ ]:
metallicity_median = MF.return_int_or_dec(np.median(data_trim["FeH"]),2)
print("Median:",metallicity_median)
print("Stars above:",np.sum(data_trim['FeH']>=metallicity_median))
print("Stars below:",np.sum(data_trim['FeH']<metallicity_median))

In [ ]:
# metal cumulative
fig,ax=plt.subplots()
ax.hist(data_trim['FeH'],cumulative=True,density=True,bins=500)
ax.grid(which='both')
ax.axvline(x=metallicity_median,color="red",label="Median")
ax.axhline(y=0.5,color="green",label="50%")
ax.set_xlabel("[Fe/H]");ax.set_ylabel("Fraction")
ax.set_xlim([data_trim["FeH"].min(),data_trim["FeH"].max()])
ax.set_ylim([0,1])
plt.legend()
plt.show()

In [ ]:
metal_poor_lowlim = None
metal_poor_highlim = metallicity_median

metal_rich_lowlim = metallicity_median
metal_rich_highlim = None

In [ ]:
# populations and labels

if metal_rich_highlim is None:
    rich_condition = data_trim["FeH"] >= metal_rich_lowlim
    label_rich = fr'[Fe/H]$~>{str(metal_rich_lowlim)}$'
else:
    rich_condition = (data_trim["FeH"] <= metal_rich_highlim)&(data_trim["FeH"] >= metal_rich_lowlim)
    label_rich = fr'${str(metal_rich_lowlim)}<~$[Fe/H]/dex$~<{str(metal_rich_highlim)}$'
    
if metal_poor_lowlim is None:
    poor_condition = data_trim["FeH"] < metal_poor_highlim
    label_poor = fr'[Fe/H]$~<{str(metal_poor_highlim)}$'
else:
    poor_condition = (data_trim["FeH"] < metal_poor_highlim)&(data_trim["FeH"] > metal_poor_lowlim)
    label_poor = fr'${str(metal_poor_lowlim)}<~$[Fe/H]/dex$~<{str(metal_poor_highlim)}$'

df_metals = [data_trim[rich_condition], data_trim[poor_condition]]
# label_rich += f" ({len(df_metals[0])})"
# label_poor += f" ({len(df_metals[1])})"

print("Rich",label_rich, len(df_metals[0]))
print("Poor",label_poor, len(df_metals[1]))

# Params

In [ ]:
# choose plotting style

blobs_bool = True
# blobs_bool = False

# dynamical_binning_bool = True
dynamical_binning_bool = False

rectangular_binning_bool = True
# rectangular_binning_bool = False
    
if True:
    if dynamical_binning_bool and rectangular_binning_bool:
        raise ValueError("Cannot have dynamical and rectangular bins, choose maximum one.")
    
    print("blobs:\t\t",blobs_bool)
    print("dynamical:\t",dynamical_binning_bool)
    print("rectangular:\t", rectangular_binning_bool)

In [ ]:
min_star_number_dyn_build = 150 if dynamical_binning_bool else None
inner_region_dict = None # initialise

if dynamical_binning_bool:
    shared_hull_bool = True
    # shared_hull_bool = False
    
    print("Working with","a shared hull" if shared_hull_bool else "separate hulls.")
    print(f"Building the dynamical bins with {min_star_number_dyn_build} minimum stars.")
    
if rectangular_binning_bool: # rectangular bins
    x_bins = 6
    
    print(f"Working with {x_bins} rectangular horizontal bins.")

In [ ]:
min_star_number_plot = 50

bootstrap_repeat = 500

In [ ]:
vel_x_variable = "r"
vel_y_variable = "l"

In [ ]:
degree_symbol = '^\circ'

In [ ]:
kinematic_symbols_dict = mapf.get_kinematic_symbols_dict(x_variable=x_variable,
                                                         y_variable=y_variable,
                                                         vel_x_variable=vel_x_variable,
                                                         vel_y_variable=vel_y_variable)

kinematic_units_dict = mapf.get_kinematic_units_dict(degree_symbol=degree_symbol)

In [ ]:
variable_symbol_dict, units_dict = mapf.get_position_symbols_and_units_dict(zabs=zabs, degree_symbol=degree_symbol)

In [ ]:
tick_step = 3

In [ ]:
# map dictionaries

xy_max = 3
z_max = 2
long_max = 15
lat_max = 15

map_min_dict = {
    "l" : -long_max,
    "b" : 0 if zabs else -lat_max,
    "d" : 6,
    "x" : -xy_max,
    "y" : -xy_max,
    "z" : -z_max,
    "R" : 0.1,
    "phi" : -180
}
map_max_dict = {
    "l" : long_max,
    "b" : lat_max,
    "d" : 10,
    "x" : xy_max,
    "y" : xy_max,
    "z" : z_max,
    "R" : 2, #maybe 1.5 judging by the xy map for 9.8-10 stars
    "phi" : 180
}
map_left_dict,map_right_dict = {},{}
for key in list(map_min_dict.keys()):
    map_left_dict[key] = map_max_dict[key] if key == 'l' else map_min_dict[key]
    map_right_dict[key] = map_min_dict[key] if key == 'l' else map_max_dict[key]

map_tick_step = {
    "l" : 3,
    "b" : 3,
    "d" : 1,
    "x" : 1,
    "y" : 1,
    "z" : 1,
    "R" : 0.5,
    "phi" : 90
}
minor_locator_dict = {
    'R': 0.25,
    'phi': 45,
    'l': 1,
    'b': 1,
    'x': 0.5,
    'y': 0.5,
    "z": 0.5,
    'd': 0.5
}
map_hstep_dict = {
    "l" : (map_max_dict['l']-map_min_dict['l'])/15,   #-10 to 10 with 15 bins gives step of 4/3
    "b" : (map_max_dict['b']-map_min_dict['b'])/10,   #0 to 10 with 10 bins gives step of 1
    "x" : 0.25,   #For -2 to 2 gives 16 bins
    "y" : 0.25,   #-2 to 2 with 16 bins gives step 0.25
    "z" : 0.25,
    "R" : (map_max_dict['R']-map_min_dict['R'])/14,
    "phi" : (map_max_dict['phi']-map_min_dict['phi'])/15  #-180 to 180 with 15 bins gives step 24
}
o_map_hstep_dict = {
    "l" : 3,
    "b" : 3,
    "x" : 0.5,
    "y" : 0.5,
    "z" : 0.5,
}
map_vstep_dict = {
    "l" : (map_max_dict['l']-map_min_dict['l'])/15,   #-10 to 10 with 15 bins gives step of 4/3
    "b" : (map_max_dict['b']-map_min_dict['b'])/10,   #0 to 10 with 10 bins gives step of 1
    "x" : 0.25,   #-2 to 2 with 16 bins gives step 0.25
    "y" : 0.25,   #-2 to 2 with 16 bins gives step 0.25
    "z" : 0.25,
    "R" : (map_max_dict['R']-map_min_dict['R'])/14,
    "phi" : (map_max_dict['phi']-map_min_dict['phi'])/15  #-180 to 180 with 15 bins gives step 24
}
o_map_vstep_dict = {
    "l" : 3,
    "b" : 3,
    "x" : 0.5,
    "y" : 0.5,
    "z" : 0.5
}
#Get the same number of "d" intervals as those of "l", so that the map has square pixels.
#The right d_step is given by l_step*Δd/Δl
map_hstep_dict["d"] = map_vstep_dict["l"]*(map_max_dict["d"]-map_min_dict["d"])/(map_max_dict["l"]-map_min_dict["l"])
map_vstep_dict["d"] = map_hstep_dict["d"]
o_map_hstep_dict["d"] = o_map_vstep_dict["l"]*(map_max_dict["d"]-map_min_dict["d"])/(map_max_dict["l"]-map_min_dict["l"])
o_map_vstep_dict["d"] = o_map_hstep_dict["d"]

In [ ]:
# save_path

save_path = general_path + "graphs/Observations/Apogee/observational_maps/"
MF.create_dir(save_path)

if not GSR:
    save_path += 'LSR/'
    MF.create_dir(save_path)

save_path += "gaia_dr3/"
MF.create_dir(save_path)

save_path += f"{x_variable}{y_variable}/"
MF.create_dir(save_path)

save_path += f"{xmin}{x_variable}{xmax}_{ymin}{y_variable}{ymax}/"
MF.create_dir(save_path)

save_path += f"{extra_min}{extra_variable}{extra_max}/" if extra_min != original_extramin or extra_max != original_extramax else "no_extra_cut"
MF.create_dir(save_path)

if metal_trim is not None:
    save_path += f"{metal_trim}metal/"
    MF.create_dir(save_path)

save_path += f"{str(metal_poor_lowlim)}to{str(metal_poor_highlim)}_{str(metal_rich_lowlim)}to{str(metal_rich_highlim)}/"
MF.create_dir(save_path)

save_path += f"min{min_star_number_plot}plot/"
MF.create_dir(save_path)

save_path += "blobs/" if blobs_bool else "no_blobs/"
MF.create_dir(save_path)

save_path_blobs = save_path

if dynamical_binning_bool:
    save_path += "dynamical_binning/"
    MF.create_dir(save_path)
    
    save_path += "shared_hull/" if shared_hull_bool else "separate_hulls/"
    MF.create_dir(save_path)
    
    save_path += f"min{min_star_number_dyn_build}dynBuild/"
    MF.create_dir(save_path)

if rectangular_binning_bool:
    save_path += "rectangular_binning/"
    MF.create_dir(save_path)
    
    save_path += f"{x_bins}xbins/"
    MF.create_dir(save_path)

save_path_inner_region = save_path

save_path += f"v{vel_x_variable}v{vel_y_variable}/"
MF.create_dir(save_path)

print("SAVING PLOTS IN\n"+save_path)

# Inner region

In [ ]:
min_x_inner = -6.7
max_x_inner = 6.5
min_y_inner = 0
max_y_inner = 6.7

if True: # Scatter only

    fig,axs=plt.subplots(ncols=2,figsize=(30,15),gridspec_kw={"wspace":0.07})
    
    cut_lw = 1
    colors = ["blue","red"]; titles = ["Young","Old"]
    for pop_index,ax in enumerate(axs):
        ax.scatter(df_metals[pop_index][x_variable],df_metals[pop_index][y_variable],s=1,color=colors[pop_index])
        ax.set_aspect("equal"); ax.set_xlim(xmax,xmin);ax.set_ylim(ymin,ymax); ax.set_xlabel(fr"${x_variable}$");ax.set_ylabel(fr"${y_variable}$")
        ax.set_title(titles[pop_index])
        
        ax.plot([min_x_inner,min_x_inner],[min_y_inner,max_y_inner],color='k',linewidth=cut_lw)
        ax.plot([max_x_inner,max_x_inner],[min_y_inner,max_y_inner],color='k',linewidth=cut_lw)
        ax.plot([min_x_inner,max_x_inner],[min_y_inner,min_y_inner],color='k',linewidth=cut_lw)
        ax.plot([min_x_inner,max_x_inner],[max_y_inner,max_y_inner],color='k',linewidth=cut_lw)
        
    filename = f"inner_region_{min_x_inner}{x_variable}{max_x_inner}_{min_y_inner}{y_variable}{max_y_inner}.png"
    plt.savefig(save_path_inner_region+filename+".png",bbox_inches="tight")
    print(filename)
    print("Saved as .png")
    plt.show()

In [ ]:
inner_region_dict = { "min_x": min_x_inner, "max_x": max_x_inner, "min_y": min_y_inner, "max_y": max_y_inner }

In [ ]:
df_hull_to_build = None
if dynamical_binning_bool:
    df_hull_to_build = [ 
        df_pop[(df_pop[x_variable]>=min_x_inner)&(df_pop[x_variable]<=max_x_inner)&(df_pop[y_variable]>=min_y_inner)&(df_pop[y_variable]<=max_y_inner)]
        for df_pop in (df_metals if not shared_hull_bool else [data_trim, data_trim])
    ]

    df_hull_to_plot = [
        df_pop[(df_pop[x_variable]>=min_x_inner)&(df_pop[x_variable]<=max_x_inner)&(df_pop[y_variable]>=min_y_inner)&(df_pop[y_variable]<=max_y_inner)]
        for df_pop in df_metals
    ]
    
if rectangular_binning_bool:
    df_rectangular = [
        df_pop[(df_pop[x_variable]>=min_x_inner)&(df_pop[x_variable]<=max_x_inner)&(df_pop[y_variable]>=min_y_inner)&(df_pop[y_variable]<=max_y_inner)]
        for df_pop in df_metals
    ]
    
    y_bins = int(x_bins * (inner_region_dict["max_y"]-inner_region_dict["min_y"])/(inner_region_dict["max_x"]-inner_region_dict["min_x"]))
    
    inner_region_dict["x_bins"] = x_bins
    inner_region_dict["y_bins"] = y_bins
    
    print(f"{x_bins} x bins")
    print(f"{y_bins} y bins")

## Visualise

In [ ]:
inner_lw = 0.5

scatter_alpha = 1
scatter_size = 0.2
scatter_color = "grey"

bin_generator_scatter_size = 5 # just for dynamical

plt.rcParams["font.size"] = 15

In [ ]:
aspect_equal = True
# aspect_equal = False

save_bool = True
# save_bool = False

In [ ]:
# build & plot

fig,axs=plt.subplots(figsize=(17,5),ncols=2,gridspec_kw={"wspace":0})

for col in range(len(df_metals)):

    ax = axs[col]

    ax.set_title([label_rich,label_poor][col])

    if dynamical_binning_bool:
        x_values_dyn_build = df_hull_to_build[col][x_variable].values
        y_values_dyn_build = df_hull_to_build[col][y_variable].values
        input_points_dyn = np.column_stack([x_values_dyn_build,y_values_dyn_build])

        pts = MultiPoint([Point(i) for i in input_points_dyn])
        mask = pts.convex_hull
        
        vor, points = dyn.generate_2Dvor(x_values_dyn_build, y_values_dyn_build, min_star_number_dyn_build)
        
        x_values_dyn_plot = df_hull_to_plot[col][x_variable].values
        y_values_dyn_plot = df_hull_to_plot[col][y_variable].values
        
        point_region = dyn.calculate_point_region(vor, x_values_dyn_plot, y_values_dyn_plot)
        computed_values = dyn.compute_values(vor.regions, df_hull_to_plot[col], point_region, compute_variable_function="counts")

        for i, cell_vertices in enumerate(vor.regions):
            polygon = vor.vertices[cell_vertices]

            shape = list(polygon.shape)
            shape[0] += 1
            p = Polygon(np.append(polygon, polygon[0]).reshape(*shape)).intersection(mask)

            ax.fill(*p.exterior.coords.xy, lw=inner_lw, edgecolor="k", facecolor="white")#cmap(norm(computed_values[i])))
            
            ax.text(points[:,0][i]+0.2,points[:,1][i]+0.15,s=int(computed_values[i]),size=10, weight="bold",zorder=20,\
                   color="k" if computed_values[i] >= min_star_number_plot else "red")
        
        ax.scatter(points[:,0],points[:,1],color="cyan",alpha=scatter_alpha,s=bin_generator_scatter_size,zorder=10)
    
    if rectangular_binning_bool:
        
        x_ranges = np.linspace(inner_region_dict["min_x"],inner_region_dict["max_x"],inner_region_dict["x_bins"]+1)
        y_ranges = np.linspace(inner_region_dict["min_y"],inner_region_dict["max_y"],inner_region_dict["y_bins"]+1)
        
        hist_range = [[inner_region_dict["min_x"],inner_region_dict["max_x"]], [inner_region_dict["min_y"],inner_region_dict["max_y"]]]
        
#         h = plt.hist2d(x=df_rectangular[col][x_variable],y=df_rectangular[col][y_variable],bins=[x_bins,y_bins],range=hist_range)
        
        for i in range(x_bins):
            
            df_x = MF.apply_cuts_to_df(df=df_rectangular[col], cuts_dict={x_variable:[x_ranges[i],x_ranges[i+1]]},\
                                       lims_dict={x_variable:"both" if i==x_bins-1 else "min"})
            
            for j in range(y_bins):
                
                df_y = MF.apply_cuts_to_df(df=df_x, cuts_dict={y_variable:[y_ranges[j],y_ranges[j+1]]},\
                                           lims_dict={y_variable:"both" if j==y_bins-1 else "min"})
                
                count = len(df_y)
                
                x_pos = (x_ranges[i] + x_ranges[i+1])/2
                y_pos = (y_ranges[j] + y_ranges[j+1])/2
                
                ax.plot([x_ranges[i],x_ranges[i]], [y_ranges[j],y_ranges[j+1]], color="k",lw=inner_lw)
                ax.plot([x_ranges[i+1],x_ranges[i+1]], [y_ranges[j],y_ranges[j+1]], color="k",lw=inner_lw)
                ax.plot([x_ranges[i],x_ranges[i+1]], [y_ranges[j],y_ranges[j]], color="k",lw=inner_lw)
                ax.plot([x_ranges[i],x_ranges[i+1]], [y_ranges[j+1],y_ranges[j+1]], color="k",lw=inner_lw)
                
                ax.text(x=x_pos,y=y_pos,s=count,size=10, weight="bold",zorder=20, color="k" if count >= min_star_number_plot else "red")
    
    ax.scatter(df_metals[col][x_variable],df_metals[col][y_variable],color=scatter_color,alpha=scatter_alpha,s=scatter_size,zorder=10)  

    if True: # axs

        ax.set_xticks(mapf.get_map_tick_range(xmin,xmax,tick_step))
        ax.set_yticks(mapf.get_map_tick_range(ymin,ymax,tick_step))

        ax.xaxis.set_minor_locator(ticker.MultipleLocator(minor_locator_dict[x_variable]))
        ax.yaxis.set_minor_locator(ticker.MultipleLocator(minor_locator_dict[y_variable]))

        if col == 0:
            ax.set_ylabel(fr"{variable_symbol_dict[y_variable]} $[{units_dict[y_variable]}]$")
        elif col != 0:
            ax.set_yticklabels([])

        ax.set_xlabel(fr"{variable_symbol_dict[x_variable]} $[{units_dict[x_variable]}]$")
        
        if aspect_equal:
            ax.set_aspect("equal")
        
        ax.set_xlim(9,-8)
        ax.set_ylim(0,8)

if True: # filename and save
    filename = f"inner_region_counts"
    print(filename)
    
    if save_bool:
        plt.savefig(save_path_inner_region+filename+".png", bbox_inches='tight',dpi=300)
        print("Saved as .png")

# Blobs

In [ ]:
if not blobs_bool:
    raise ValueError("`blobs_bool` is set to False")

In [ ]:
def plot_scatter_with_circles(x_values, y_values, circle_radii, circle_centres, almost_inside_lim=0.03, min_star_number=min_star_number_plot,\
                              xlims=(xmax,xmin), ylims=(ymin,ymax),figsize=(30,10),inner_region_dict=None,title=None,filepath="",save_bool=False):
    
    fig,ax=plt.subplots(figsize=figsize)
    
    for i, (radius,centre) in enumerate(zip(circle_radii,circle_centres)):
        circle = np.array(PH.get_ellipse_coords(radius, ratio=1, centre=centre)).T

        points_inside = MF.in_circle(x_values, y_values, centre[0], centre[1], radius)
        
        almost_inside = MF.in_circle(x_values,y_values,centre[0],centre[1],radius+almost_inside_lim)
        ax.scatter(x_values[(~points_inside)&almost_inside],y_values[(~points_inside)&almost_inside],color="red",s=10)

        lw = 2 if sum(points_inside) < min_star_number else 0.5
        
        if sum(points_inside) < min_star_number:
            yellow_circle = PH.get_ellipse_coords(radius=radius-0.03,centre=centre)
            ax.plot(yellow_circle[0],yellow_circle[1], color="yellow", lw=1)
        ax.plot(circle[:,0],circle[:,1], color="k", lw=0.5)
        
        ax.text(x = centre[0],y=centre[1]-1.2*radius,s=str(i+1),size=10,color="k")
        ax.text(x = centre[0]-radius/np.sqrt(2), y=centre[1]+radius/np.sqrt(2),s=str(sum(points_inside)),size=10,color="red")

    ax.scatter(x_values,y_values,color="grey",s=1,zorder=10)
        
    if inner_region_dict is not None:
        ax.plot([inner_region_dict["min_x"], inner_region_dict["min_x"], inner_region_dict["max_x"], inner_region_dict["max_x"]],
                [inner_region_dict["min_y"], inner_region_dict["max_y"], inner_region_dict["max_y"], inner_region_dict["min_y"]],
                color="red")
    
    if True: # ax
        ax.set_xticks(np.arange(int(xmin),int(xmax)+1,1))
        ax.set_yticks(np.arange(int(ymin),int(ymax)+1,1))
        ax.tick_params(labeltop=True, labelright=True)

        x_label = fr"{variable_symbol_dict[x_variable]} $[{units_dict[x_variable]}]$"
        y_label = fr"{variable_symbol_dict[y_variable]} $[{units_dict[y_variable]}]$"

        ax.set_xlabel(x_label)
        ax.set_ylabel(y_label)
        
        ax.set_aspect('equal')
        ax.set_xlim(xmax,xmin);ax.set_ylim(ymin,ymax)
    
    if title is not None: ax.set_title(title)
        
    if save_bool:
        if filename == "":
            raise ValueError("Please give a filename to save the plot.")
        
        plt.savefig(filepath+".png", bbox_inches='tight',dpi=300)
        print("Saved:",filepath+".png")
    
    plt.show()

In [ ]:
def get_all_circle_radii_and_centres(pop_idx):
    
    if pop_idx == 0:
        idx =     [     1,          2,           3,          4,           5,          6,          7,          8,          9,         10,           11,         12,         13,       14,         15,         16,           17]
        centres = [[-10.05,4],[-9.975,11.96],[-5,7.98],[-4.94,11.94],[0.07,7.96],[4.96,7.91],[5.19,12.15],[9.9,3.95],[10,11.95],[4.888,10.156],[7.47,2.1],[7.09,7.04],[8.51,8.46],[10,1.97],[9.75,0.51], [-10.2,0.2], [0.1,11.97]]
        radii =   [    0.85,       0.8,        0.88,       0.83,         0.95,       0.915,      0.95,       0.85,       0.87,        0.9,        0.96,       0.9,         0.9,      0.55,      0.52,         0.55,        0.7]
        
    else:
        idx =     [     1,          2,         3,          4,          5,          6,          7,         8,       9,       10,        11,        12,         13,        14,          15,        16,     17,        18,           19]
        centres = [[-10.1,3.95],[-10,12],[-4.95,7.98],[-7.6,12.35],[-4.9,12],[-3.6,12.2],[0.03,7.96],[0,11.95],[5,7.9],[5.27,12.1],[9.9,3.95],[10,11.95],[4.9,10.13],[7.56,2.09],[7.08,7.03],[8.5,8.5],[10,2],[7.15,12.235], [3.34,9.09]]
        radii =   [    0.85,       0.85,      0.88,       0.9,         0.9,       0.8,        0.95,      0.91,    0.9,      1,       0.85,        0.87,      0.92,     0.88,        0.92,       0.92,    0.51,     0.9,          0.82]
    
    return radii, centres

def get_circle_radii_and_centres_with_more_than_n_stars(df_pop,pop_idx,min_star_number=30,x_variable="l",y_variable="b"):
    
    all_radii, all_centres = get_all_circle_radii_and_centres(pop_idx)
    minN_radii, minN_centres = [],[]
    
    x_values = df_pop[x_variable].values
    y_values = df_pop[y_variable].values
    
    for radius,centre in zip(all_radii,all_centres):
        
        points_inside = MF.in_circle(x_values, y_values, centre[0], centre[1], radius)
        
        if sum(points_inside) >= min_star_number:
            minN_radii.append(radius)
            minN_centres.append(centre)
            
    return minN_radii, minN_centres

In [ ]:
almost_inside_lim = 0.05

# save_bool = True
save_bool = False

pop_idx = 1

In [ ]:
# plot

plt.rcParams["font.size"] = 15

x_values = df_metals[pop_idx][x_variable].values
y_values = df_metals[pop_idx][y_variable].values

circle_radii, circle_centres = get_all_circle_radii_and_centres(pop_idx)

filepath = save_path_blobs + f"blobs_counts_%s"%["young","old"][pop_idx]

plot_scatter_with_circles(x_values, y_values, circle_radii, circle_centres, almost_inside_lim=almost_inside_lim, inner_region_dict=inner_region_dict, 
                          title=["Young","Old"][pop_idx],save_bool=save_bool,filepath=filepath, min_star_number=min_star_number_plot)

# Functions

In [ ]:
def compute_mean(df, **kws):
    return np.mean(df[kws["vel"]].values)

def compute_std(df, **kws):
    return np.std(df[kws["vel"]].values)

def compute_correlation(df, **kws):
    return CV.calculate_correlation(df[kws["vx"]].values,df[kws["vy"]].values)

def compute_anisotropy(df, **kws):
    return CV.calculate_anisotropy(df[kws["vx"]].values,df[kws["vy"]].values)

def compute_tilt_abs(df, **kws):
    return CV.calculate_tilt(df[kws["vx"]].values,df[kws["vy"]].values,absolute=True)

def compute_error(df, **kws):
    return CE.get_std_bootstrap(vx=df[kws["vx"]].values,vy=df[kws["vy"]].values, function=kws["function"], tilt=kws["tilt"], absolute=kws["absolute"], repeat=kws["repeat"])

def compute_fractional_error(df, **kws):
    function = kws["function"]
    
    error = CE.get_std_bootstrap(vx=df[kws["vx"]].values,vy=df[kws["vy"]].values, function=function, tilt=kws["tilt"], absolute=kws["absolute"], repeat=kws["repeat"])
    value = function(df[kws["vx"]].values,df[kws["vy"]].values)
    
    return error/abs(value)

In [ ]:
def get_compute_variable_error_function(map_variable):
    if map_variable == "correlation":
        return CV.calculate_correlation
    elif map_variable == "anisotropy":
        return CV.calculate_anisotropy
    elif map_variable == "tilt_abs":
        return CV.calculate_tilt
    else:
        raise ValueError(f"map variable `{map_variable}` not recognised")

def get_compute_variable_function(map_variable):
    if map_variable in ["mean_vx","mean_vy"]:
        return compute_mean
    elif map_variable in ["std_vx","std_vy"]:
        return compute_std
    elif map_variable == "correlation":
        return compute_correlation
    elif map_variable == "anisotropy":
        return compute_anisotropy
    elif map_variable == "tilt_abs":
        return compute_tilt_abs
    elif "fractionalerror" in map_variable:
        return compute_fractional_error
    elif "error" in map_variable:
        return compute_error
    else:
        raise ValueError(f"map variable `{map_variable}` not recognised")

def is_tilt(map_variable):
    return "tilt" in map_variable or "vertex" in map_variable # this considers X_abs too
        
def get_kws_for_error(map_variable, bootstrap_repeat=500):
    if "error" not in map_variable:
        raise ValueError(f"Expected the variable to be a variable error, but found `{map_variable}`.")
    
    error_string = "_fractionalerror" if "_fractionalerror" in map_variable else "_error"
    kinematic_variable = map_variable.replace(error_string,"")
    
    error_kws = {
        "function": get_compute_variable_error_function(kinematic_variable),
         "tilt": is_tilt(kinematic_variable),
         "absolute": is_tilt(kinematic_variable) and "_abs" in kinematic_variable,
         "repeat": bootstrap_repeat
    }
    return error_kws
        
def get_kws_and_compute_variable_function(map_variable, bootstrap_repeat=500):

    if map_variable in ["mean_vx","std_vx"]:
        kws={"vel":f"v{vel_x_variable}"}
    elif map_variable in ["mean_vy","std_vy"]:
        kws={"vel":f"v{vel_y_variable}" if vel_y_variable !="\phi" else "vphi"}
    else:
        kws={"vx":f"v{vel_x_variable}","vy":f"v{vel_y_variable}" if vel_y_variable !="\phi" else "vphi"}

    compute_variable = get_compute_variable_function(map_variable)
    
    if "error" in map_variable:
        error_kws = get_kws_for_error(map_variable, bootstrap_repeat)
        kws = {**kws, **error_kws} # combine dictionaries
        
    return kws, compute_variable

In [ ]:
class GetMapValuesAndMinMaxExtendLists():
    def __init__(self, blobs_bool, dynamical_binning_bool, rectangular_binning_bool, min_star_number_plot,\
                 bootstrap_repeat=500,x_variable="l",y_variable="b",\
                 min_star_number_dyn_build=None, inner_region_dict=None):
        
        self.blobs_bool = blobs_bool
        self.dynamical_binning_bool = dynamical_binning_bool
        self.rectangular_binning_bool = rectangular_binning_bool
        self.min_star_number_plot = min_star_number_plot
        self.bootstrap_repeat = bootstrap_repeat
        self.x_variable = x_variable
        self.y_variable = y_variable
        
        self.plotting_params_given = False
        
        if inner_region_dict is not None:
            self.inner_region_dict = inner_region_dict
        elif rectangular_binning_bool or dynamical_binning_bool:
            raise ValueError("Working in the inner region but `inner_region_dict` was not passed")
        
        if min_star_number_dyn_build is not None:
            self.min_star_number_dyn_build = min_star_number_dyn_build
        elif dynamical_binning_bool:
            raise ValueError("`dynamical_binning_bool` is True but `min_star_number_dyn_build` was not passed")
    
    def get_map_name(self):
        name = "dyn" if self.dynamical_binning_bool else ""
        name += "rect" if self.rectangular_binning_bool else ""
        name += "blobs" if self.blobs_bool else ""
        
        return name
    
    def set_plotting_params(self, blob_lw=0.5, inner_lw=0.5, lowN_lw=2, lowN_edgecolor="yellow"):
        
        self.blob_lw = blob_lw
        self.inner_lw = inner_lw
        self.lowN_edgecolor = lowN_edgecolor
        self.lowN_lw = lowN_lw
        
        self.plotting_params_given = True
        
    def get_new_vminvmax(self,current_vmin,current_vmax,values):
        return min(current_vmin,np.nanmin(values)), max(current_vmax,np.nanmax(values))
    
    def select_inner_region(self, df):
        return df[
                    (df[self.x_variable]>=self.inner_region_dict["min_x"])&
                    (df[self.x_variable]<=self.inner_region_dict["max_x"])&
                    (df[self.y_variable]>=self.inner_region_dict["min_y"])&
                    (df[self.y_variable]<=self.inner_region_dict["max_y"])
                ]
        
    def validate_compute_method(self, ax,cmap,norm,compute_variable,compute_variable_kwgs):
        
        if ax is not None:
            if not self.plotting_params_given:
                raise ValueError("Invoke the `set_plotting_params` method before proceeding with plotting.")
            if cmap is None or norm is None:
                raise ValueError("Plotting requires passing `cmap` and `norm`.")
                
        if compute_variable != "number" and compute_variable_kwgs is None:
            raise ValueError("Computing a variable different than 'number' requires passing `compute_variable_kwgs`")
    
    def compute_and_plot_rectangular_bins_for_single_pop(self, df_pop, compute_variable, compute_variable_kwgs=None,apply_inner_cut=False,\
                                                         ax=None, cmap=None, norm=None,below_min_number_show_bool=True):
        
        self.validate_compute_method(ax,cmap,norm,compute_variable,compute_variable_kwgs)
        
        if apply_inner_cut:
            df_pop = self.select_inner_region(df_pop)
        
        x_bins,y_bins = self.inner_region_dict["x_bins"], self.inner_region_dict["y_bins"]

        x_ranges = np.linspace(self.inner_region_dict["min_x"],self.inner_region_dict["max_x"],x_bins+1)
        y_ranges = np.linspace(self.inner_region_dict["min_y"],self.inner_region_dict["max_y"],y_bins+1)

        counts = np.zeros(shape=(x_bins,y_bins))
        values = np.zeros(shape=(x_bins,y_bins))
        
        lowN_vertices_values_list = []
        
        def get_vertices(x_ranges,y_ranges,i,j):
            return np.array([
                [x_ranges[i], y_ranges[j]],
                [x_ranges[i+1], y_ranges[j]],
                [x_ranges[i+1], y_ranges[j+1]],
                [x_ranges[i], y_ranges[j+1]],
            ])
        
        for i in range(x_bins):

            df_x = MF.apply_cuts_to_df(df=df_pop, cuts_dict={self.x_variable:[x_ranges[i],x_ranges[i+1]]},\
                                       lims_dict={self.x_variable:"both" if i==x_bins-1 else "min"})

            for j in range(y_bins):

                df_y = MF.apply_cuts_to_df(df=df_x, cuts_dict={self.y_variable:[y_ranges[j],y_ranges[j+1]]},\
                                           lims_dict={self.y_variable:"both" if j==y_bins-1 else "min"})

                counts[i,j] = len(df_y)
                values[i,j] = compute_variable(df_y, **compute_variable_kwgs) if compute_variable != "number" else len(df_y)
        
                if ax is not None: # plot
                    
                    if not below_min_number_show_bool and counts[i,j] < self.min_star_number_plot:
                        continue
                    
                    vertices = get_vertices(x_ranges,y_ranges,i,j)
                    
                    if counts[i,j] >= self.min_star_number_plot:
                        ax.fill(vertices[:,0],vertices[:,1], lw=self.inner_lw, facecolor=cmap(norm(values[i,j])), edgecolor="k")
                    else:
                        lowN_vertices_values_list.append({
                            "vertices": vertices,
                            "value": values[i,j]
                        })
        
        if ax is not None and below_min_number_show_bool: # plot lowN separately so the edges don't overlap
            for l in lowN_vertices_values_list:
                ax.fill(l["vertices"][:,0],l["vertices"][:,1], lw=self.lowN_lw, facecolor=cmap(norm(l["value"])), edgecolor=self.lowN_edgecolor)

        lowN_values = values[counts < self.min_star_number_plot]        
        values = values[counts >= self.min_star_number_plot]
    
        return values, lowN_values
    
    def compute_and_plot_blobs_for_single_pop(self,df_pop,pop_idx, compute_variable, compute_variable_kwgs=None,ax=None, cmap=None, norm=None,\
                                             below_min_number_show_bool=True):
        
        self.validate_compute_method(ax,cmap,norm,compute_variable,compute_variable_kwgs)
        
        all_x_values = df_pop[self.x_variable].values
        all_y_values = df_pop[self.y_variable].values
        
        if not below_min_number_show_bool:
            circle_radii, circle_centres = get_circle_radii_and_centres_with_more_than_n_stars(df_pop=df_pop,pop_idx=pop_idx,\
                                                                                               min_star_number=self.min_star_number_plot,\
                                                                                               x_variable=self.x_variable,y_variable=self.y_variable)
        else:
            circle_radii, circle_centres = get_all_circle_radii_and_centres(pop_idx=pop_idx)

        counts = []
        values = []

        for radius,centre in zip(circle_radii,circle_centres):
            circle = np.array(PH.get_ellipse_coords(radius, ratio=1, centre=centre, tilt=0)).T

            points_inside = MF.in_circle(all_x_values, all_y_values, centre[0], centre[1], radius)
            
            number = sum(points_inside)
            counts.append(number)
            
            value = compute_variable(df_pop.iloc[points_inside], **compute_variable_kwgs) if compute_variable != "number" else number
            values.append(value)
            
            if ax is not None:
                edge_color = self.lowN_edgecolor if number < self.min_star_number_plot else "k"
                lw = self.lowN_lw if number < self.min_star_number_plot else self.blob_lw

                ax.fill(circle[:,0],circle[:,1], edgecolor=edge_color, lw=lw, facecolor=cmap(norm(value)))

        counts = np.array(counts)
        values = np.array(values)

        lowN_values = values[counts < self.min_star_number_plot]
        values = values[counts >= self.min_star_number_plot]
            
        return values, lowN_values
        
    def compute_and_plot_dynamical_bins_for_single_pop(self,df_pop,df_build_pop,compute_variable,compute_variable_kwgs=None,apply_inner_cut=False,\
                                                       ax=None,cmap=None,norm=None,below_min_number_show_bool=True):
        
        self.validate_compute_method(ax,cmap,norm,compute_variable,compute_variable_kwgs)
        
        if apply_inner_cut:
            df_pop = self.select_inner_region(df_pop)
            df_build_pop = self.select_inner_region(df_build_pop)
        
        x_values_dyn_build = df_build_pop[self.x_variable].values
        y_values_dyn_build = df_build_pop[self.y_variable].values
        input_points_dyn = np.column_stack([x_values_dyn_build,y_values_dyn_build])

        pts = MultiPoint([Point(i) for i in input_points_dyn])
        mask = pts.convex_hull

        vor, points = dyn.generate_2Dvor(x_values_dyn_build, y_values_dyn_build, self.min_star_number_dyn_build)

        x_values_dyn_plot = df_pop[x_variable].values
        y_values_dyn_plot = df_pop[y_variable].values
        point_region = dyn.calculate_point_region(vor, x_values_dyn_plot, y_values_dyn_plot)

        counts = dyn.compute_values(vor.regions, df_pop, point_region, compute_variable_function="counts")

        if compute_variable != "number":
            values = dyn.compute_values(vor.regions, df_pop, point_region, compute_variable_function=compute_variable, **compute_variable_kwgs)
        else:
            values = counts
        
        if ax is not None: # plot
            
            lowN_xy = []
            lowN_values = []

            for i, cell_vertices in enumerate(vor.regions):

                if not below_min_number_show_bool and counts[i] < self.min_star_number_plot:
                    continue

                polygon = vor.vertices[cell_vertices]

                shape = list(polygon.shape)
                shape[0] += 1
                p = Polygon(np.append(polygon, polygon[0]).reshape(*shape)).intersection(mask)

                if counts[i] >= self.min_star_number_plot:
                    ax.fill(*p.exterior.coords.xy, lw=self.inner_lw, facecolor=cmap(norm(values[i])), edgecolor="k")
                else:
                    lowN_xy.append(p.exterior.coords.xy)
                    lowN_values.append(values[i])

            for xy,val in zip(lowN_xy,lowN_values): # plot lowN after the rest so edges are not superposed
                ax.fill(*xy, lw=self.lowN_lw, facecolor=cmap(norm(val)), edgecolor=self.lowN_edgecolor)
                
        lowN_values = values[counts < self.min_star_number_plot]
        values = values[counts >= self.min_star_number_plot]
        
        return values, lowN_values, points
    
    def get_vminvmaxextend_lists(self,df_all,map_variable_list, below_min_number_show_bool,below_min_number_for_vminvmax_bool,\
                                 df_hull_build=None,bootstrap_repeat=500, x_variable="l", y_variable="b", hardcode=False):
        
        if self.dynamical_binning_bool and df_hull_build is None:
            raise ValueError("`dynamical_binning_bool` is True but `df_hull_build` was not given")
        
        vmin_list,vmax_list,extend_list = np.full(len(map_variable_list),None), np.full(len(map_variable_list),None), np.full(len(map_variable_list),None)
        
        if self.dynamical_binning_bool or self.rectangular_binning_bool:
            df_inner = [self.select_inner_region(df) for df in df_all]
            
            if self.dynamical_binning_bool:
                df_hull_build_inner = [self.select_inner_region(df) for df in df_hull_build]
        
        for row,variable in enumerate(map_variable_list):

            if variable != "number":
                compute_variable_kwgs, compute_variable = get_kws_and_compute_variable_function(variable,bootstrap_repeat=bootstrap_repeat)
            else:
                compute_variable = "number"
                compute_variable_kwgs = None

            vmin,vmax = float("inf"),float("-inf")
            lowN_vmin, lowN_vmax = float("inf"),float("-inf")

            for col in range(len(df_all)):
                
                values_to_concat, lowN_values_to_concat = [],[]
                
                if self.blobs_bool:
                    blob_values,lowN_blob_values = self.compute_and_plot_blobs_for_single_pop(df_pop=df_all[col],pop_idx=col,compute_variable=compute_variable,\
                                                                                              compute_variable_kwgs=compute_variable_kwgs)
                    
                    values_to_concat.append(blob_values); lowN_values_to_concat.append(lowN_blob_values)
                
                if self.rectangular_binning_bool:
                    rect_values, lowN_rect_values = self.compute_and_plot_rectangular_bins_for_single_pop(df_pop=df_inner[col], compute_variable=compute_variable,\
                                                                                                          compute_variable_kwgs=compute_variable_kwgs)
                    
                    values_to_concat.append(rect_values.flatten()); lowN_values_to_concat.append(lowN_rect_values.flatten())
                
                if self.dynamical_binning_bool:
                    dyn_values, lowN_dyn_values, *_ = self.compute_and_plot_dynamical_bins_for_single_pop(df_pop=df_inner[col],df_build_pop=df_hull_build_inner[col],\
                                                                                                      compute_variable=compute_variable,\
                                                                                                      compute_variable_kwgs=compute_variable_kwgs)
                    
                    values_to_concat.append(dyn_values); lowN_values_to_concat.append(lowN_dyn_values)
                
                vmin,vmax = self.get_new_vminvmax(vmin,vmax,np.concatenate(values_to_concat))
                
                if below_min_number_show_bool and below_min_number_for_vminvmax_bool:
                    vmin,vmax = self.get_new_vminvmax(vmin,vmax,np.concatenate(lowN_values_to_concat))
                elif below_min_number_show_bool:
                    lowN_vmin,lowN_vmax = self.get_new_vminvmax(lowN_vmin, lowN_vmax, np.concatenate(lowN_values_to_concat))
                
            set_hardcode = False
            if hardcode:
                if "fractionalerror" in variable:
                    hard_vmin,hard_vmax = 0,1.5
                    vmin_list[row],vmax_list[row],extend_list[row] = hard_vmin,hard_vmax,PH.get_cbar_extend(hard_vmin,hard_vmax,vmin,vmax)
                    set_hardcode = True
            if not set_hardcode:
                vmin_list[row],vmax_list[row],extend_list[row] = vmin,vmax,"neither"

            if below_min_number_show_bool and not below_min_number_for_vminvmax_bool:
                lowN_vmin = min(vmin,lowN_vmin)
                lowN_vmax = max(vmax,lowN_vmax)

                lowN_vmin_extend = "min" if lowN_vmin < vmin else "neither"
                lowN_vmax_extend = "max" if lowN_vmax > vmax else "neither"

                extend_list[row] = PH.cbar_extend_list_union([extend_list[row],lowN_vmin_extend, lowN_vmax_extend])

        return vmin_list,vmax_list,extend_list

In [ ]:
map_helper = GetMapValuesAndMinMaxExtendLists(blobs_bool=blobs_bool,dynamical_binning_bool=dynamical_binning_bool,rectangular_binning_bool=rectangular_binning_bool,\
                                              min_star_number_plot=min_star_number_plot,bootstrap_repeat=bootstrap_repeat,x_variable=x_variable,y_variable=y_variable,\
                                              inner_region_dict=inner_region_dict, min_star_number_dyn_build=min_star_number_dyn_build)

# Star number

In [ ]:
vmin_list,vmax_list,extend_list = map_helper.get_vminvmaxextend_lists(df_all=df_metals,df_hull_build=df_hull_to_build,below_min_number_show_bool=True,\
                                                                      below_min_number_for_vminvmax_bool=False,map_variable_list=["number"])

nmin,nmax,cbar_extend = vmin_list[0],vmax_list[0],extend_list[0] 

cbar_ticks = np.arange(min_star_number_plot,nmax,50).astype(int)
print(cbar_ticks)

In [ ]:
cmap = mplcmaps["viridis"]
cmap.set_under("silver")

norm = plt.Normalize(min_star_number_plot,nmax)

In [ ]:
plt.rcParams["font.size"] = 20

In [ ]:
map_helper.set_plotting_params(blob_lw=0.5,lowN_edgecolor="k",lowN_lw=0.5,inner_lw=0.5)

In [ ]:
bin_generator_scatter_bool = False
scatter_bool = True

scatter_alpha = 1
scatter_size = 0.2
scatter_color = "red"

In [ ]:
# aspect_equal = True
aspect_equal = False

save_bool = True
# save_bool = False

In [ ]:
# plot

fig,(*axs,cax)=plt.subplots(figsize=(17,4.95),ncols=3,gridspec_kw={"wspace":0,"width_ratios":[1,1,0.05]})

for col,ax in enumerate(axs):
    
    ax.set_title([label_rich,label_poor][col])
    
    if blobs_bool:
        map_helper.compute_and_plot_blobs_for_single_pop(df_pop=df_metals[col],pop_idx=col,compute_variable="number",\
                                                         ax=ax,cmap=cmap,norm=norm,below_min_number_show_bool=True)
    
    if dynamical_binning_bool:
        _,_,points = map_helper.compute_and_plot_dynamical_bins_for_single_pop(df_pop=df_hull_to_plot[col],df_build_pop=df_hull_to_build[col],compute_variable="number",\
                                                         below_min_number_show_bool=True,ax=ax,cmap=cmap,norm=norm)
    
    if rectangular_binning_bool:
        map_helper.compute_and_plot_rectangular_bins_for_single_pop(df_pop=df_rectangular[col],compute_variable="number",\
                                                                   below_min_number_show_bool=True,ax=ax,cmap=cmap,norm=norm)
    
    if scatter_bool:
        ax.scatter(df_metals[col][x_variable],df_metals[col][y_variable],color=scatter_color,alpha=scatter_alpha,s=scatter_size,zorder=10)
    if bin_generator_scatter_bool:
        ax.scatter(points[:,0],points[:,1],color="cyan",alpha=bin_generator_scatter_alpha,s=bin_generator_scatter_size,zorder=10)

    if True: # axs

        ax.set_xticks(mapf.get_map_tick_range(xmin,xmax,tick_step))
        ax.set_yticks(mapf.get_map_tick_range(ymin,ymax,tick_step))

        ax.xaxis.set_minor_locator(ticker.MultipleLocator(minor_locator_dict[x_variable]))
        ax.yaxis.set_minor_locator(ticker.MultipleLocator(minor_locator_dict[y_variable]))

        if col == 0:
            ax.set_ylabel(fr"{variable_symbol_dict[y_variable]} $[{units_dict[y_variable]}]$")
        else:
            ax.set_yticklabels([])

        ax.set_xlabel(fr"{variable_symbol_dict[x_variable]} $[{units_dict[x_variable]}]$")
        
        if aspect_equal:
            ax.set_aspect("equal")
        
        ax.set_xlim(xmax,xmin)
        ax.set_ylim(ymin,ymax)
        
if True: # colorbar
    cbar = plt.colorbar(cm.ScalarMappable(norm=norm,cmap=cmap), cax=cax, extend=cbar_extend)#,shrink=0.7)
    cbar.set_label(r"$N$",labelpad=15,rotation=0)
    cbar.set_ticks(cbar_ticks)
    
if True: # save
    filename = map_helper.get_map_name()
    
    filename += "_counts"
    
    if save_bool:
        print(save_path+filename)
        for fileformat in [".png",".pdf"]:
            plt.savefig(save_path_inner_region+filename+fileformat, bbox_inches='tight',dpi=300)
            print("Saved:",fileformat)
    else:
        print(filename)

    plt.show()

# Double plot

In [ ]:
# CHOOSE

# doublemap_variable_list = [["anisotropy","correlation","tilt_abs"], ["anisotropy_error","correlation_error","tilt_abs_error"]]; map_name="anicorr"
doublemap_variable_list = [["mean_vx","mean_vy"],["std_vx","std_vy"]]; map_name = "velmeanstd"

left_maps = doublemap_variable_list[0]
right_maps = doublemap_variable_list[1]

In [ ]:
sharing_cbar_bool = True
# sharing_cbar_bool = False

if sharing_cbar_bool:
    
    shared_cbar_variables = []
    
    shared_cbar_variables.append(["std_vx","std_vy"])
        
    print("Sharing all of the following map pairs (when they exists):\n")
    for shared in shared_cbar_variables:
        print(shared)

In [ ]:
# Ensure that the boostrap errors computed to get the vmin and vmax are the same as they are then computed when producing the plot
np.random.seed(1)

In [ ]:
# below_min_number_show_bool = True
below_min_number_show_bool = False

below_min_number_for_vminvmax_bool = True # only takes effect if below_min_number_show_bool is True
# below_min_number_for_vminvmax_bool = False

In [ ]:
# calculate vmin,vmax

vmin_list_left, vmax_list_left, extend_list_left = map_helper.get_vminvmaxextend_lists(df_all=df_metals,df_hull_build=df_hull_to_build,\
                                                                      below_min_number_show_bool=below_min_number_show_bool,\
                                                                      below_min_number_for_vminvmax_bool=below_min_number_for_vminvmax_bool,\
                                                                      map_variable_list=left_maps)

vmin_list_right, vmax_list_right, extend_list_right = map_helper.get_vminvmaxextend_lists(df_all=df_metals,df_hull_build=df_hull_to_build,\
                                                                      below_min_number_show_bool=below_min_number_show_bool,\
                                                                      below_min_number_for_vminvmax_bool=below_min_number_for_vminvmax_bool,\
                                                                      map_variable_list=right_maps)

if sharing_cbar_bool:
    mapf.share_vminvmax_given_vminvmax_lists(left_maps, shared_cbar_variables, vmin_list_left, vmax_list_left)
    mapf.share_vminvmax_given_vminvmax_lists(right_maps, shared_cbar_variables, vmin_list_right, vmax_list_right)

vmin_double_list = [vmin_list_left, vmin_list_right]
vmax_double_list = [vmax_list_left, vmax_list_right]
extend_double_list = [extend_list_left, extend_list_right]

In [ ]:
# scatter_bool = True
scatter_bool = False

if scatter_bool:
    scatter_alpha = 0.5
    scatter_size = 0.1
    scatter_color="grey"

In [ ]:
map_helper.set_plotting_params(blob_lw=0.5,inner_lw=0.5,lowN_edgecolor="yellow",lowN_lw=2)

In [ ]:
# aspect_equal = True
aspect_equal = False

save_bool = True
# save_bool = False

In [ ]:
def get_doublemap_gridspec_params(variables, n_rows):
    if variables == "lb":
        if n_rows == 2:
            fig_size = 7
            fig_aspect_ratio = 2.29*(xmax-xmin)/(ymax-ymin)
            central_space = 0.51
            cbar_width = 0.05
#             fig_size = 7
#             fig_aspect_ratio = 2.3*(x_max-x_min)/(y_max-y_min)
#             central_space = 0.51
#             cbar_width = 0.05
            cbar_ticksize = plt.rcParams['ytick.major.size'] - 2
            plt.rcParams['font.size'] = 26
        elif n_rows == 3:
            fig_size = 10
            fig_aspect_ratio = 4.52/3*(xmax-xmin)/(ymax-ymin)
            central_space = 0.47
            cbar_width = 0.05
            cbar_ticksize = plt.rcParams['ytick.major.size'] - 2
            plt.rcParams['font.size'] = 26
        else:
            raise NotImplementedError(f"Not implemented for {n_rows} rows.")
    else:
        raise NotImplementedError(f"Not implemented for {variables}.")
            
    return fig_size, fig_aspect_ratio, central_space, cbar_width, cbar_ticksize

In [ ]:
# Plot

if True: # define fig and axes
    n_rows = len(left_maps)
    n_cols = 7
    
    fig_size, fig_aspect_ratio, central_space, cbar_width, cbar_ticksize = get_doublemap_gridspec_params(x_variable+y_variable, n_rows)
    grid = gridspec.GridSpec(n_rows,n_cols,width_ratios=[1, 1, cbar_width, central_space, 1, 1, cbar_width], hspace=0,wspace=0)
    fig = plt.figure(figsize=(fig_aspect_ratio*fig_size,fig_size))
    
    leftaxs = np.reshape([fig.add_subplot(grid[i,j]) for i in range(n_rows) for j in range(n_cols) if j in [0,1]], (n_rows,2))
    leftaxs_cbars = [fig.add_subplot(grid[i,j]) for i in range(n_rows) for j in range(n_cols) if j == 2]

    rightaxs = np.reshape([fig.add_subplot(grid[i,j]) for i in range(n_rows) for j in range(n_cols) if j in [4,5]], (n_rows,2))
    rightaxs_cbars = [fig.add_subplot(grid[i,j]) for i in range(n_rows) for j in range(n_cols) if j == 6]

    axes = [leftaxs,rightaxs]
    axes_cbar = [leftaxs_cbars,rightaxs_cbars]

for block in range(len(doublemap_variable_list)):
    for row in range(n_rows):
        variable = doublemap_variable_list[block][row]

        kws, compute_variable = get_kws_and_compute_variable_function(variable)

        vmin, vmax, cbar_extend = vmin_double_list[block][row], vmax_double_list[block][row], extend_double_list[block][row]

        cmap = PH.choose_cmap(vmin,vmax)
        norm = plt.Normalize(vmin,vmax)

        for col in range(len(df_metals)):
            
            ax = axes[block][row][col]
            
            if row == 0: ax.set_title([label_rich,label_poor][col])

            if dynamical_binning_bool:
                map_helper.compute_and_plot_dynamical_bins_for_single_pop(df_pop=df_hull_to_plot[col],df_build_pop=df_hull_to_build[col],ax=ax,\
                                                                          below_min_number_show_bool=below_min_number_show_bool,cmap=cmap,norm=norm,\
                                                                          compute_variable=compute_variable,compute_variable_kwgs=kws)

            if blobs_bool:
                map_helper.compute_and_plot_blobs_for_single_pop(df_pop=df_metals[col],pop_idx=col,ax=ax,below_min_number_show_bool=below_min_number_show_bool,\
                                                                 compute_variable=compute_variable, compute_variable_kwgs=kws,cmap=cmap,norm=norm)
            
            if rectangular_binning_bool:
                map_helper.compute_and_plot_rectangular_bins_for_single_pop(df_pop=df_rectangular[col],ax=ax,below_min_number_show_bool=below_min_number_show_bool,\
                                                                            compute_variable=compute_variable, compute_variable_kwgs=kws,cmap=cmap,norm=norm)
            

            if scatter_bool: 
                ax.scatter(df_metals[col][x_variable],df_metals[col][y_variable],color=scatter_color,alpha=scatter_alpha,s=scatter_size,zorder=10)
                
            if True: # axs
                
                x_ticks = mapf.get_map_tick_range(xmin,xmax,tick_step)
                y_ticks = mapf.get_map_tick_range(ymin,ymax,tick_step)
                
                ax.set_xticks(x_ticks)
                ax.set_yticks(y_ticks)
                
                ax.xaxis.set_minor_locator(ticker.MultipleLocator(minor_locator_dict[x_variable]))
                ax.yaxis.set_minor_locator(ticker.MultipleLocator(minor_locator_dict[y_variable]))
                
                if block == 0 and col == 0:
                    ax.set_ylabel(fr"{variable_symbol_dict[y_variable]} $[{units_dict[y_variable]}]$")
                elif col != 0:
                    ax.set_yticklabels([])
                    
                if col==0 and row != n_rows-1:
                    if x_variable+y_variable=="lb" and ymax - max(y_ticks) <= 1:
                        ax.set_yticklabels([None]+y_ticks[1:])

                if row == n_rows-1: 
                    ax.set_xlabel(fr"{variable_symbol_dict[x_variable]} $[{units_dict[x_variable]}]$")
                else:
                    ax.set_xticklabels([])

                if aspect_equal:
                    ax.set_aspect("equal")
                    
                ax.set_xlim(xmax,xmin)
                ax.set_ylim(ymin,ymax)
        
        if True: # colorbar
            cax = axes_cbar[block][row]
            
            cbar = plt.colorbar(cm.ScalarMappable(norm=norm,cmap=cmap), cax=cax, label=kinematic_symbols_dict[variable]+kinematic_units_dict[variable],\
                               extend=cbar_extend, extendfrac=0.1)#,shrink=0.7)
            cbar.ax.locator_params(nbins=6)
            
for block in range(len(doublemap_variable_list)): # remove overlapping ticks
    for row in range(n_rows - 1):
        current_vminvmax = [vmin_double_list[block][row],vmax_double_list[block][row]]
        next_vminvmax = [vmin_double_list[block][row+1],vmax_double_list[block][row+1]]
        
        current_cbar_extend, next_cbar_extend = extend_double_list[block][row], extend_double_list[block][row+1]
        
        if current_cbar_extend in ["min","both"] or next_cbar_extend in ["max","both"]:
            continue
        
        mapf.remove_overlapping_ticks(axes_cbar[block][row], axes_cbar[block][row+1], current_vminvmax, next_vminvmax, which="bottom",cbar_frac=14)

if True: # filename and save
    fig.align_labels()
    
    below_min_n_str = ""
    if below_min_number_show_bool:
        if not below_min_number_for_vminvmax_bool:
            below_min_n_str = "_noLowNvminvmax"
    else:
        below_min_n_str = "_noLowNshow"
        
    scatter_string = "_scatter" if scatter_bool else ""
    edges_string = "_noEdges" if map_helper.inner_lw == 0 and map_helper.blob_lw == 0 else ""
    
    maps_string = '_'+map_name + ("_sharing" if sharing_cbar_bool and mapf.any_map_pair_is_shared(doublemap_variable_list, shared_cbar_variables) else "")
    
    filename = f"{x_variable}{y_variable}_{map_helper.get_map_name()}_doublemap{maps_string}{below_min_n_str}{scatter_string}{edges_string}"
    
    _all_maps = MF.flatten_list(list(doublemap_variable_list))
    if any("error" in map_var for map_var in _all_maps) and bootstrap_repeat != 500:
        filename += f"_boot{bootstrap_repeat}"
    
    print(save_path)
    print(filename)
    
    if save_bool:
        for fileformat in [".png",".pdf"]:
            plt.savefig(save_path+filename+fileformat, bbox_inches='tight',dpi=300)
            print("Saved:",fileformat)

plt.show()

# Single plot

In [ ]:
# CHOOSE

# map_variable_list = ["mean_vx","mean_vy","varx","vary"]
map_variable_list = ["anisotropy","correlation","tilt_abs"]
# map_variable_list = ["anisotropy_error","correlation_error","tilt_abs_error"]
# map_variable_list = ["anisotropy","anisotropy_error","correlation","correlation_error","tilt_abs","tilt_abs_error"]

for variable in map_variable_list: print(variable)

In [ ]:
vmin_list, vmax_list = get_vminvmax_list_for_lb_blobhull_maps(df_metals, df_hull, map_variable_list=map_variable_list, min_star_number_dyn=min_star_number_dyn,
                                                              bootstrap_repeat=500, x_variable=x_variable, y_variable=y_variable)

In [ ]:
fig_size = 20
cbar_width = 0.035

def get_fig_aspect_ratio(n_rows):
    if n_rows == 3:
        return 1.1
    elif n_rows == 4:
        return 1
    else:
        raise NotImplementedError(f"Not implemented for {n_rows} rows")

In [ ]:
scatter_bool = True
# scatter_bool = False

# aspect_equal = True
aspect_equal = False

# save_bool = True
save_bool = False

In [ ]:
# Plot

n_rows = len(map_variable_list)
n_cols = len(df_metals) + 1 # includes cbar

grid = gridspec.GridSpec(n_rows,3,width_ratios=[1, 1, cbar_width], hspace=0,wspace=0)

fig = plt.figure(figsize=(get_fig_aspect_ratio(n_rows)*fig_size,fig_size))
axes = np.array([fig.add_subplot(grid[i,j]) for i in range(n_rows) for j in range(n_cols)]).reshape((n_rows,n_cols))

for row,variable in enumerate(map_variable_list):

    kws, compute_variable = get_kws_and_compute_variable_function(variable)
    
    vmin, vmax = vmin_list[row], vmax_list[row]
    
    cmap = PH.choose_cmap(vmin,vmax)
    norm = plt.Normalize(vmin,vmax)

    row_axs = axes[row]
    
    for col in range(len(df_metals)+1):

        ax = row_axs[col]

        if col == 2:
            plt.colorbar(cm.ScalarMappable(norm=norm,cmap=cmap), cax=ax, label=kinematic_symbols_dict[variable]+kinematic_units_dict[variable])#,shrink=0.7)
            continue

        if True: # calculate dynamical hull
            x_values_dyn = df_hull[col][x_variable].values
            y_values_dyn = df_hull[col][y_variable].values
            input_points_dyn = np.column_stack([x_values_dyn,y_values_dyn])

            pts = MultiPoint([Point(i) for i in input_points_dyn])
            mask = pts.convex_hull

            vor, points = dyn.generate_2Dvor(x_values_dyn, y_values_dyn, min_star_number_dyn)
            point_region = dyn.calculate_point_region(vor, x_values_dyn, y_values_dyn)
            computed_values = dyn.compute_values(vor.regions, df_hull[col], point_region, compute_variable_function=compute_variable,**kws)
            
            for i, cell_vertices in enumerate(vor.regions):
                polygon = vor.vertices[cell_vertices]

                shape = list(polygon.shape)
                shape[0] += 1
                p = Polygon(np.append(polygon, polygon[0]).reshape(*shape)).intersection(mask)

                ax.fill(*p.exterior.coords.xy, lw=0, facecolor=cmap(norm(computed_values[i])))

        if True: # blobs
            all_x_values = df_metals[col][x_variable].values
            all_y_values = df_metals[col][y_variable].values

            circle_radii, circle_centres = get_circle_radii_and_centres(pop_idx=col)
            
            for radius,centre in zip(circle_radii,circle_centres):
                circle = np.array(PH.get_ellipse_coords(radius, ratio=1, centre=centre, tilt=0)).T

                points_inside = MF.in_circle(all_x_values, all_y_values, centre[0], centre[1], radius)

                value = compute_variable(df_metals[col].iloc[points_inside], **kws)

                edge_color = "yellow" if sum(points_inside) < min_star_number_blobs else "k"
                lw = 2 if sum(points_inside) < min_star_number_blobs else 0

                ax.fill(circle[:,0],circle[:,1], edgecolor=edge_color, lw=lw, facecolor=cmap(norm(value)))
        
        # ax.plot([min_l_dyn,min_l_dyn,max_l_dyn,max_l_dyn],[min_b_dyn,max_b_dyn,max_b_dyn,min_b_dyn],color="red")
        
        if scatter_bool:
            ax.scatter(all_x_values,all_y_values,color="grey",s=1,zorder=10)
            ax.scatter(points[:,0],points[:,1],color="cyan",alpha=0.5,s=5,zorder=10)
        
if True: #axs
    x_label = fr"{variable_symbol_dict[x_variable]} $[{units_dict[x_variable]}]$"
    y_label = fr"{variable_symbol_dict[y_variable]} $[{units_dict[y_variable]}]$"

    x_ticks = get_map_tick_range(x_variable)
    y_ticks = get_map_tick_range(y_variable)
    
    for ax in axes.flatten():
        if ax in axes[:,2]:
            continue
        
        ax.set_xticklabels(x_ticks if ax in axes[-1] else [])
        ax.set_yticklabels(y_ticks if ax in axes[:,0] else [])
        
        ax.set_xlabel(x_label if ax in axes[-1] else "")
        ax.set_ylabel(y_label if ax in axes[:,0] else "")

        if aspect_equal:
            ax.set_aspect("equal")

        axes[0,0].set_title(label_rich)
        axes[0,1].set_title(label_poor)
        
        ax.set_xticks(x_ticks)
        ax.set_yticks(y_ticks)
        
        ax.set_xlim(lmax,lmin)
        ax.set_ylim(bmin,bmax)
        
if save_bool:
    extra_var_string = f"{dmin}d{dmax}"
    
    filename = f"{x_variable}{y_variable}_dynmap_{extra_var_string}_{map_variables[-2]}_{map_variables[-1]}_min{min_star_number_dyn}dyn_min{min_star_number_blobs}blobs"

    if any("error" in map_var for map_var in map_variables):
        filename += f"_boot{bootstrap_repeat}"

    if scatter_bool:
        filename += "_scatter"

    plt.savefig(save_path+filename+".png", bbox_inches='tight',dpi=300)
    plt.savefig(save_path+filename+".pdf", bbox_inches='tight')
    print(save_path+filename)
plt.show()